In [1]:
# Setup et imports
import pandas as pd
import re
import xml.etree.ElementTree as ET
from xml.dom import minidom
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import math

print("🔧 Pipeline Vekta Enhanced V2 - Améliorations Avancées")
print("=" * 60)


🔧 Pipeline Vekta Enhanced V2 - Améliorations Avancées


In [2]:
# 1. Configuration avancée avec entités détaillées
class EnhancedValidationConfig:
    # Seuils de confiance renforcés
    CONFIDENCE_THRESHOLD_HIGH = 0.95    # Excellence requise
    CONFIDENCE_THRESHOLD_MEDIUM = 0.75  # Très bonne correspondance
    CONFIDENCE_THRESHOLD_LOW = 0.75     # Rejet en dessous
    
    # Entités requises avec patterns améliorés et exemples
    REQUIRED_ENTITIES = {
        'duration': {
            'pattern': r'\d+\s*(min|h|s|sec|secondes?|minutes?|heures?)',
            'examples': ['10min', '1h', '30s', '45 minutes'],
            'description': 'Durée des intervalles ou de la séance'
        },
        'repetitions': {
            'pattern': r'\d+\s*[x*×]\s*',
            'examples': ['3x', '5*', '8×', '4 fois'],
            'description': 'Nombre de répétitions'
        },
        'intensity': {
            'pattern': r'(VO2|seuil|tempo|aerobic|sprint|endurance|threshold|anaerobic|neuromuscular|\d+%)',
            'examples': ['VO2max', 'tempo', 'seuil', 'aerobic', '85%', 'sprint'],
            'description': 'Zone d\'intensité ou pourcentage'
        },
        'recovery': {
            'pattern': r'(recup|rec|recovery|repos)\s*\d+',
            'examples': ['5min recup', '2min repos', '90s recovery'],
            'description': 'Temps de récupération entre intervalles'
        },
        'structure': {
            'pattern': r'(pyramide|pyramid|series?|blocs?|escalier)',
            'examples': ['pyramide', 'série', '2 blocs', 'escalier'],
            'description': 'Structure complexe de l\'entraînement'
        }
    }
    
    # Pondérations pour scoring avancé
    SCORING_WEIGHTS = {
        'exact_match': 2.0,      # Correspondance exacte
        'zone_match': 1.5,       # Zone d'entraînement
        'duration_match': 1.2,   # Durée
        'repetition_match': 1.2, # Répétitions
        'intensity_match': 1.0,  # Intensité
        'structure_match': 1.8,  # Structure complexe
        'variation_bonus': 0.3   # Bonus variation linguistique
    }

print("✅ Configuration avancée chargée")
print(f"   Seuils: Excellence ≥{EnhancedValidationConfig.CONFIDENCE_THRESHOLD_HIGH}")
print(f"   Entités surveillées: {len(EnhancedValidationConfig.REQUIRED_ENTITIES)}")
print(f"   Critères de scoring: {len(EnhancedValidationConfig.SCORING_WEIGHTS)}")


✅ Configuration avancée chargée
   Seuils: Excellence ≥0.95
   Entités surveillées: 5
   Critères de scoring: 7


In [3]:
# 2. Corpus étendu avec structures complexes
enhanced_corpus_data = [
    # === STRUCTURES SIMPLES ===
    {
        'id': 0, 'description': '1h endurance allure aerobic tranquille', 'zone': 'Aerobic',
        'structure': {'type': 'continuous', 'duration': 60}, 'power_zone': 2, 'intensity': 65,
        'complexity': 'simple',
        'variations': ['1h endur aerobic', '60min endurance facile', '1 heure aerobic tranquil']
    },
    {
        'id': 1, 'description': '3x 10 min tempo sweet spot avec 5min recup', 'zone': 'Tempo',
        'structure': {'type': 'intervals', 'reps': 3, 'duration': 10, 'recovery': 5}, 'power_zone': 3, 'intensity': 85,
        'complexity': 'simple',
        'variations': ['3x10min tempo 5minrec', '3*10min sweet spot', '3 fois 10 minutes tempo']
    },
    
    # === STRUCTURES COMPLEXES - PYRAMIDES ===
    {
        'id': 10, 'description': 'pyramide 1-2-3-4-3-2-1min VO2max avec 1min recup', 'zone': 'VO2max',
        'structure': {'type': 'pyramid', 'pattern': [1,2,3,4,3,2,1], 'recovery': 1}, 'power_zone': 5, 'intensity': 112,
        'complexity': 'complex',
        'variations': ['pyramide 1234321 VO2', 'pyramid 1-2-3-4-3-2-1min', 'pyramide VO2max classique']
    },
    {
        'id': 11, 'description': 'pyramide tempo 5-10-15-10-5min avec 5min recup', 'zone': 'Tempo',
        'structure': {'type': 'pyramid', 'pattern': [5,10,15,10,5], 'recovery': 5}, 'power_zone': 3, 'intensity': 87,
        'complexity': 'complex',
        'variations': ['pyramide tempo 5-10-15', 'pyramid sweet spot', 'pyramide 5/10/15/10/5']
    },
    
    # === STRUCTURES COMPLEXES - SÉRIES ===
    {
        'id': 20, 'description': '2 series de 4x3min VO2max avec 90s recup et 5min entre series', 'zone': 'VO2max',
        'structure': {'type': 'series', 'sets': 2, 'reps': 4, 'duration': 3, 'recovery_intra': 1.5, 'recovery_inter': 5}, 
        'power_zone': 5, 'intensity': 115, 'complexity': 'complex',
        'variations': ['2*(4x3min VO2)', '2 blocs 4x3min VO2max', '2 series 4x3min avec 90s']
    },
    {
        'id': 21, 'description': '3 series de 3x5min seuil avec 2min recup et 8min entre series', 'zone': 'Threshold',
        'structure': {'type': 'series', 'sets': 3, 'reps': 3, 'duration': 5, 'recovery_intra': 2, 'recovery_inter': 8},
        'power_zone': 4, 'intensity': 100, 'complexity': 'complex',
        'variations': ['3*(3x5min seuil)', '3 blocs 3x5min FTP', '3 series seuil 5min']
    },
    
    # === STRUCTURES SPÉCIALISÉES ===
    {
        'id': 30, 'description': 'over-under 8x3min 95-105% FTP avec 2min recup', 'zone': 'Threshold',
        'structure': {'type': 'over_under', 'reps': 8, 'duration': 3, 'low': 95, 'high': 105, 'recovery': 2},
        'power_zone': 4, 'intensity': 100, 'complexity': 'complex',
        'variations': ['over under 95-105%', '8x3min over-under', 'sur-sous seuil']
    },
    {
        'id': 31, 'description': 'micro-intervalles 15x1min ON/OFF VO2max', 'zone': 'VO2max',
        'structure': {'type': 'micro_intervals', 'reps': 15, 'duration': 1, 'recovery': 1, 'on_off': True},
        'power_zone': 5, 'intensity': 115, 'complexity': 'complex',
        'variations': ['15x1min ON OFF', 'micro intervalles VO2', '15*1min on off']
    }
]

# Expansion avec variations
all_enhanced_entries = []
for workout in enhanced_corpus_data:
    all_enhanced_entries.append(workout)
    if 'variations' in workout:
        for i, variation in enumerate(workout['variations']):
            var_entry = workout.copy()
            var_entry['id'] = f"{workout['id']}_v{i+1}"
            var_entry['description'] = variation
            var_entry['is_variation'] = True
            all_enhanced_entries.append(var_entry)

enhanced_corpus_df = pd.DataFrame(all_enhanced_entries)

print(f"📚 Corpus Enhanced V2: {len(enhanced_corpus_df)} entraînements")
print(f"   Principaux: {len(enhanced_corpus_data)}")
print(f"   Variations: {len(enhanced_corpus_df) - len(enhanced_corpus_data)}")
print(f"   Complexité: {enhanced_corpus_df['complexity'].value_counts().to_dict()}")


📚 Corpus Enhanced V2: 32 entraînements
   Principaux: 8
   Variations: 24
   Complexité: {'complex': 24, 'simple': 8}


In [4]:
# 3. Validateur Enhanced avec messages d'erreur détaillés
class EnhancedValidator:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.config = EnhancedValidationConfig()
    
    def detect_missing_entities_detailed(self, query: str) -> Tuple[Dict, Dict]:
        """Détection détaillée des entités manquantes avec exemples"""
        query_lower = query.lower()
        entities_analysis = {}
        
        for entity_type, entity_config in self.config.REQUIRED_ENTITIES.items():
            pattern = entity_config['pattern']
            matches = re.findall(pattern, query_lower, re.IGNORECASE)
            
            entities_analysis[entity_type] = {
                'found': len(matches) > 0,
                'matches': matches,
                'examples': entity_config['examples'],
                'description': entity_config['description']
            }
        
        missing_entities = {k: v for k, v in entities_analysis.items() if not v['found']}
        return entities_analysis, missing_entities
    
    def calculate_advanced_similarity(self, query: str, description: str, workout_data: Dict) -> Tuple[float, Dict]:
        """Calcul de similarité avancé avec pondération"""
        query_lower = query.lower()
        desc_lower = description.lower()
        
        score_components = {}
        
        # 1. Correspondance exacte (bonus majeur)
        if query_lower.strip() == desc_lower.strip():
            score_components['exact_match'] = self.config.SCORING_WEIGHTS['exact_match']
        else:
            score_components['exact_match'] = 0
        
        # 2. Zone d'entraînement
        zone_keywords = {
            'aerobic': ['aerobic', 'endurance', 'z2'],
            'tempo': ['tempo', 'sweet', 'spot', 'z3'],
            'threshold': ['threshold', 'seuil', 'ftp', 'z4'],
            'vo2max': ['vo2', 'vo2max', 'z5'],
            'anaerobic': ['anaerobic', 'sprint', 'z6'],
            'neuromuscular': ['neuromuscular', 'neuro', 'max']
        }
        
        zone_score = 0
        workout_zone = workout_data.get('zone', '').lower()
        for zone, keywords in zone_keywords.items():
            if zone in workout_zone:
                for keyword in keywords:
                    if keyword in query_lower:
                        zone_score += 0.3
        score_components['zone_match'] = min(zone_score, 1.0) * self.config.SCORING_WEIGHTS['zone_match']
        
        # 3. Durées et nombres
        query_numbers = re.findall(r'\d+', query)
        desc_numbers = re.findall(r'\d+', description)
        
        duration_score = 0
        for num in query_numbers:
            if num in desc_numbers:
                duration_score += 0.2
        score_components['duration_match'] = min(duration_score, 1.0) * self.config.SCORING_WEIGHTS['duration_match']
        
        # 4. Répétitions (3x, 5x, etc.)
        rep_patterns_query = re.findall(r'\d+\s*[x*×]', query_lower)
        rep_patterns_desc = re.findall(r'\d+\s*[x*×]', desc_lower)
        
        rep_score = 0
        for rep in rep_patterns_query:
            rep_num = re.findall(r'\d+', rep)[0]
            for desc_rep in rep_patterns_desc:
                desc_num = re.findall(r'\d+', desc_rep)[0]
                if rep_num == desc_num:
                    rep_score += 0.5
        score_components['repetition_match'] = min(rep_score, 1.0) * self.config.SCORING_WEIGHTS['repetition_match']
        
        # 5. Structures complexes (bonus majeur)
        structure_keywords = ['pyramide', 'pyramid', 'serie', 'series', 'bloc', 'escalier', 'over', 'under', 'micro']
        structure_score = 0
        
        for keyword in structure_keywords:
            if keyword in query_lower and keyword in desc_lower:
                structure_score += 0.4
                # Bonus pour complexité
                if workout_data.get('complexity') == 'complex':
                    structure_score += 0.3
        
        score_components['structure_match'] = min(structure_score, 1.0) * self.config.SCORING_WEIGHTS['structure_match']
        
        # Score total
        total_score = sum(score_components.values())
        
        # Normalisation (score max théorique = somme des poids)
        max_possible_score = sum(self.config.SCORING_WEIGHTS.values())
        normalized_score = min(total_score / max_possible_score, 0.99)
        
        return normalized_score, score_components
    
    def search_corpus_enhanced(self, query: str) -> Tuple[Optional[Dict], float, Dict]:
        """Recherche avancée dans le corpus avec scoring détaillé"""
        best_match = None
        best_score = 0.0
        best_components = {}
        
        for idx, workout in self.corpus_df.iterrows():
            score, components = self.calculate_advanced_similarity(
                query, workout['description'], workout.to_dict()
            )
            
            # Bonus pour variations exactes
            if workout.get('is_variation', False):
                score += self.config.SCORING_WEIGHTS['variation_bonus']
            
            if score > best_score:
                best_score = score
                best_match = workout.to_dict()
                best_components = components
        
        return best_match, min(best_score, 0.99), best_components
    
    def generate_detailed_error_message(self, query: str, missing_entities: Dict, best_match: Dict, confidence: float) -> str:
        """Génération de messages d'erreur détaillés et constructifs"""
        message_parts = []
        
        # En-tête avec niveau de confiance
        if confidence < 0.3:
            message_parts.append(f"❌ Requête non reconnue (confiance: {confidence:.3f})")
        else:
            message_parts.append(f"⚠️ Correspondance insuffisante (confiance: {confidence:.3f})")
        
        # Entités manquantes détaillées
        if missing_entities:
            message_parts.append("\\n🔍 ENTITÉS MANQUANTES:")
            for entity_type, entity_info in missing_entities.items():
                examples = ", ".join(entity_info['examples'][:3])
                message_parts.append(f"   • {entity_info['description']}")
                message_parts.append(f"     Exemples: {examples}")
        
        # Suggestions d'amélioration
        message_parts.append("\\n💡 SUGGESTIONS:")
        
        if best_match:
            message_parts.append(f"   • Essayez: '{best_match['description']}'")
            
            # Suggestions spécifiques selon les entités manquantes
            if 'duration' in missing_entities:
                message_parts.append("   • Précisez la durée: '10min', '1h', '30s'")
            if 'repetitions' in missing_entities:
                message_parts.append("   • Ajoutez le nombre: '3x', '5*', '8 fois'")
            if 'intensity' in missing_entities:
                message_parts.append("   • Spécifiez l'intensité: 'tempo', 'VO2max', '85%'")
            if 'recovery' in missing_entities:
                message_parts.append("   • Indiquez la récupération: '5min recup', '2min repos'")
            if 'structure' in missing_entities:
                message_parts.append("   • Précisez la structure: 'pyramide', 'série', 'blocs'")
        
        return "\\n".join(message_parts)
    
    def validate_query_enhanced(self, query: str) -> Dict:
        """Validation complète avec analyse détaillée"""
        # Analyse des entités
        entities_analysis, missing_entities = self.detect_missing_entities_detailed(query)
        
        # Recherche dans le corpus
        best_match, confidence, score_components = self.search_corpus_enhanced(query)
        
        # Fallback si aucun match
        if best_match is None:
            best_match = self.corpus_df.iloc[0].to_dict()
            confidence = 0.1
        
        # Détermination de la validité
        if confidence >= self.config.CONFIDENCE_THRESHOLD_HIGH:
            is_valid = True
            message = f"✅ Excellence (confiance: {confidence:.3f}) - Génération autorisée"
        elif confidence >= self.config.CONFIDENCE_THRESHOLD_MEDIUM:
            is_valid = True
            message = f"⚠️ Bonne correspondance (confiance: {confidence:.3f}) - Génération avec avertissement"
        else:
            is_valid = False
            message = self.generate_detailed_error_message(query, missing_entities, best_match, confidence)
        
        return {
            'query': query,
            'confidence': confidence,
            'score_components': score_components,
            'corpus_match': best_match,
            'is_valid': is_valid,
            'message': message,
            'entities_analysis': entities_analysis,
            'missing_entities': list(missing_entities.keys()),
            'missing_entities_detailed': missing_entities
        }

enhanced_validator = EnhancedValidator(enhanced_corpus_df)
print("🔍 Validateur Enhanced V2 initialisé")
print("   ✅ Messages d'erreur détaillés")
print("   ✅ Scoring avancé avec pondération")
print("   ✅ Support structures complexes")


🔍 Validateur Enhanced V2 initialisé
   ✅ Messages d'erreur détaillés
   ✅ Scoring avancé avec pondération
   ✅ Support structures complexes


In [5]:
# 4. Fonction de démonstration Enhanced
def demonstrate_enhanced_pipeline_v2(query: str):
    """Démonstration du pipeline Enhanced V2 avec analyses détaillées"""
    
    print(f"\n🔍 REQUÊTE: '{query}'")
    print("=" * 80)
    
    # Validation enhanced
    validation = enhanced_validator.validate_query_enhanced(query)
    
    # Affichage des résultats
    print(f"📊 ANALYSE DE CONFIANCE:")
    print(f"   Score global: {validation['confidence']:.3f}")
    
    if 'score_components' in validation and validation['score_components']:
        print(f"   Composants du score:")
        for component, score in validation['score_components'].items():
            if score > 0:
                print(f"     • {component}: {score:.3f}")
    
    print(f"\n📋 RÉSULTAT:")
    print(validation['message'])
    
    # Analyse des entités si rejet
    if not validation['is_valid'] and validation['missing_entities']:
        print(f"\n🔍 ANALYSE DES ENTITÉS:")
        for entity_type, details in validation['missing_entities_detailed'].items():
            print(f"   ❌ {entity_type}: {details['description']}")
            print(f"      Exemples: {', '.join(details['examples'][:2])}")
    
    # Correspondance trouvée
    if validation['corpus_match']:
        match = validation['corpus_match']
        print(f"\n🎯 MEILLEURE CORRESPONDANCE:")
        print(f"   Description: {match['description']}")
        print(f"   Zone: {match['zone']}")
        print(f"   Complexité: {match.get('complexity', 'N/A')}")
        if 'structure' in match:
            print(f"   Structure: {match['structure']}")
    
    return validation

print("🚀 Fonction de démonstration Enhanced V2 prête")
print("   Usage: demonstrate_enhanced_pipeline_v2('votre requête')")


🚀 Fonction de démonstration Enhanced V2 prête
   Usage: demonstrate_enhanced_pipeline_v2('votre requête')


In [6]:
# Test 1: Structure complexe - Pyramide VO2max
demonstrate_enhanced_pipeline_v2("pyramide VO2max 1-2-3-4-3-2-1min")



🔍 REQUÊTE: 'pyramide VO2max 1-2-3-4-3-2-1min'
📊 ANALYSE DE CONFIANCE:
   Score global: 0.733
   Composants du score:
     • zone_match: 0.900
     • duration_match: 1.200
     • structure_match: 1.800

📋 RÉSULTAT:
⚠️ Correspondance insuffisante (confiance: 0.733)\n\n🔍 ENTITÉS MANQUANTES:\n   • Nombre de répétitions\n     Exemples: 3x, 5*, 8×\n   • Temps de récupération entre intervalles\n     Exemples: 5min recup, 2min repos, 90s recovery\n\n💡 SUGGESTIONS:\n   • Essayez: 'pyramide 1-2-3-4-3-2-1min VO2max avec 1min recup'\n   • Ajoutez le nombre: '3x', '5*', '8 fois'\n   • Indiquez la récupération: '5min recup', '2min repos'

🔍 ANALYSE DES ENTITÉS:
   ❌ repetitions: Nombre de répétitions
      Exemples: 3x, 5*
   ❌ recovery: Temps de récupération entre intervalles
      Exemples: 5min recup, 2min repos

🎯 MEILLEURE CORRESPONDANCE:
   Description: pyramide 1-2-3-4-3-2-1min VO2max avec 1min recup
   Zone: VO2max
   Complexité: complex
   Structure: {'type': 'pyramid', 'pattern': [1, 2, 3

{'query': 'pyramide VO2max 1-2-3-4-3-2-1min',
 'confidence': 0.7333333333333332,
 'score_components': {'exact_match': 0,
  'zone_match': 0.8999999999999999,
  'duration_match': 1.2,
  'repetition_match': 0.0,
  'structure_match': 1.8},
 'corpus_match': {'id': 10,
  'description': 'pyramide 1-2-3-4-3-2-1min VO2max avec 1min recup',
  'zone': 'VO2max',
  'structure': {'type': 'pyramid',
   'pattern': [1, 2, 3, 4, 3, 2, 1],
   'recovery': 1},
  'power_zone': 5,
  'intensity': 112,
  'complexity': 'complex',
  'variations': ['pyramide 1234321 VO2',
   'pyramid 1-2-3-4-3-2-1min',
   'pyramide VO2max classique'],
  'is_variation': nan},
 'is_valid': False,
 'message': "⚠️ Correspondance insuffisante (confiance: 0.733)\\n\\n🔍 ENTITÉS MANQUANTES:\\n   • Nombre de répétitions\\n     Exemples: 3x, 5*, 8×\\n   • Temps de récupération entre intervalles\\n     Exemples: 5min recup, 2min repos, 90s recovery\\n\\n💡 SUGGESTIONS:\\n   • Essayez: 'pyramide 1-2-3-4-3-2-1min VO2max avec 1min recup'\\n   •

In [7]:
# Test 2: Structure complexe - Séries
demonstrate_enhanced_pipeline_v2("2 series de 4x3min VO2max")



🔍 REQUÊTE: '2 series de 4x3min VO2max'
📊 ANALYSE DE CONFIANCE:
   Score global: 0.773
   Composants du score:
     • zone_match: 0.900
     • duration_match: 0.960
     • repetition_match: 0.600
     • structure_match: 1.800

📋 RÉSULTAT:
⚠️ Bonne correspondance (confiance: 0.773) - Génération avec avertissement

🎯 MEILLEURE CORRESPONDANCE:
   Description: 2 series de 4x3min VO2max avec 90s recup et 5min entre series
   Zone: VO2max
   Complexité: complex
   Structure: {'type': 'series', 'sets': 2, 'reps': 4, 'duration': 3, 'recovery_intra': 1.5, 'recovery_inter': 5}


{'query': '2 series de 4x3min VO2max',
 'confidence': 0.7733333333333332,
 'score_components': {'exact_match': 0,
  'zone_match': 0.8999999999999999,
  'duration_match': 0.96,
  'repetition_match': 0.6,
  'structure_match': 1.8},
 'corpus_match': {'id': 20,
  'description': '2 series de 4x3min VO2max avec 90s recup et 5min entre series',
  'zone': 'VO2max',
  'structure': {'type': 'series',
   'sets': 2,
   'reps': 4,
   'duration': 3,
   'recovery_intra': 1.5,
   'recovery_inter': 5},
  'power_zone': 5,
  'intensity': 115,
  'complexity': 'complex',
  'variations': ['2*(4x3min VO2)',
   '2 blocs 4x3min VO2max',
   '2 series 4x3min avec 90s'],
  'is_variation': nan},
 'is_valid': True,
 'message': '⚠️ Bonne correspondance (confiance: 0.773) - Génération avec avertissement',
 'entities_analysis': {'duration': {'found': True,
   'matches': ['s', 'min'],
   'examples': ['10min', '1h', '30s', '45 minutes'],
   'description': 'Durée des intervalles ou de la séance'},
  'repetitions': {'foun

In [8]:
# Test 3: Requête incomplète - Messages d'erreur détaillés
demonstrate_enhanced_pipeline_v2("entraînement pyramide")



🔍 REQUÊTE: 'entraînement pyramide'
📊 ANALYSE DE CONFIANCE:
   Score global: 0.500
   Composants du score:
     • structure_match: 1.800

📋 RÉSULTAT:
⚠️ Correspondance insuffisante (confiance: 0.500)\n\n🔍 ENTITÉS MANQUANTES:\n   • Durée des intervalles ou de la séance\n     Exemples: 10min, 1h, 30s\n   • Nombre de répétitions\n     Exemples: 3x, 5*, 8×\n   • Zone d'intensité ou pourcentage\n     Exemples: VO2max, tempo, seuil\n   • Temps de récupération entre intervalles\n     Exemples: 5min recup, 2min repos, 90s recovery\n\n💡 SUGGESTIONS:\n   • Essayez: 'pyramide 1-2-3-4-3-2-1min VO2max avec 1min recup'\n   • Précisez la durée: '10min', '1h', '30s'\n   • Ajoutez le nombre: '3x', '5*', '8 fois'\n   • Spécifiez l'intensité: 'tempo', 'VO2max', '85%'\n   • Indiquez la récupération: '5min recup', '2min repos'

🔍 ANALYSE DES ENTITÉS:
   ❌ duration: Durée des intervalles ou de la séance
      Exemples: 10min, 1h
   ❌ repetitions: Nombre de répétitions
      Exemples: 3x, 5*
   ❌ intensity: 

{'query': 'entraînement pyramide',
 'confidence': 0.49999999999999994,
 'score_components': {'exact_match': 0,
  'zone_match': 0.0,
  'duration_match': 0.0,
  'repetition_match': 0.0,
  'structure_match': 1.8},
 'corpus_match': {'id': 10,
  'description': 'pyramide 1-2-3-4-3-2-1min VO2max avec 1min recup',
  'zone': 'VO2max',
  'structure': {'type': 'pyramid',
   'pattern': [1, 2, 3, 4, 3, 2, 1],
   'recovery': 1},
  'power_zone': 5,
  'intensity': 112,
  'complexity': 'complex',
  'variations': ['pyramide 1234321 VO2',
   'pyramid 1-2-3-4-3-2-1min',
   'pyramide VO2max classique'],
  'is_variation': nan},
 'is_valid': False,
 'message': "⚠️ Correspondance insuffisante (confiance: 0.500)\\n\\n🔍 ENTITÉS MANQUANTES:\\n   • Durée des intervalles ou de la séance\\n     Exemples: 10min, 1h, 30s\\n   • Nombre de répétitions\\n     Exemples: 3x, 5*, 8×\\n   • Zone d'intensité ou pourcentage\\n     Exemples: VO2max, tempo, seuil\\n   • Temps de récupération entre intervalles\\n     Exemples: 5

In [9]:
# Test 4: Requête très vague - Suggestions constructives
demonstrate_enhanced_pipeline_v2("intervalles")



🔍 REQUÊTE: 'intervalles'
📊 ANALYSE DE CONFIANCE:
   Score global: 0.300
   Composants du score:

📋 RÉSULTAT:
⚠️ Correspondance insuffisante (confiance: 0.300)\n\n🔍 ENTITÉS MANQUANTES:\n   • Durée des intervalles ou de la séance\n     Exemples: 10min, 1h, 30s\n   • Nombre de répétitions\n     Exemples: 3x, 5*, 8×\n   • Zone d'intensité ou pourcentage\n     Exemples: VO2max, tempo, seuil\n   • Temps de récupération entre intervalles\n     Exemples: 5min recup, 2min repos, 90s recovery\n   • Structure complexe de l'entraînement\n     Exemples: pyramide, série, 2 blocs\n\n💡 SUGGESTIONS:\n   • Essayez: '1h endurance allure aerobic tranquille'\n   • Précisez la durée: '10min', '1h', '30s'\n   • Ajoutez le nombre: '3x', '5*', '8 fois'\n   • Spécifiez l'intensité: 'tempo', 'VO2max', '85%'\n   • Indiquez la récupération: '5min recup', '2min repos'\n   • Précisez la structure: 'pyramide', 'série', 'blocs'

🔍 ANALYSE DES ENTITÉS:
   ❌ duration: Durée des intervalles ou de la séance
      Exemple

{'query': 'intervalles',
 'confidence': 0.3,
 'score_components': {'exact_match': 0,
  'zone_match': 0.0,
  'duration_match': 0.0,
  'repetition_match': 0.0,
  'structure_match': 0.0},
 'corpus_match': {'id': 0,
  'description': '1h endurance allure aerobic tranquille',
  'zone': 'Aerobic',
  'structure': {'type': 'continuous', 'duration': 60},
  'power_zone': 2,
  'intensity': 65,
  'complexity': 'simple',
  'variations': ['1h endur aerobic',
   '60min endurance facile',
   '1 heure aerobic tranquil'],
  'is_variation': nan},
 'is_valid': False,
 'message': "⚠️ Correspondance insuffisante (confiance: 0.300)\\n\\n🔍 ENTITÉS MANQUANTES:\\n   • Durée des intervalles ou de la séance\\n     Exemples: 10min, 1h, 30s\\n   • Nombre de répétitions\\n     Exemples: 3x, 5*, 8×\\n   • Zone d'intensité ou pourcentage\\n     Exemples: VO2max, tempo, seuil\\n   • Temps de récupération entre intervalles\\n     Exemples: 5min recup, 2min repos, 90s recovery\\n   • Structure complexe de l'entraînement\\

In [10]:
# Test 5: Structure spécialisée - Over-under
demonstrate_enhanced_pipeline_v2("over-under 8x3min FTP")



🔍 REQUÊTE: 'over-under 8x3min FTP'
📊 ANALYSE DE CONFIANCE:
   Score global: 0.670
   Composants du score:
     • zone_match: 0.450
     • duration_match: 0.480
     • repetition_match: 0.600
     • structure_match: 1.800

📋 RÉSULTAT:
⚠️ Correspondance insuffisante (confiance: 0.670)\n\n🔍 ENTITÉS MANQUANTES:\n   • Zone d'intensité ou pourcentage\n     Exemples: VO2max, tempo, seuil\n   • Temps de récupération entre intervalles\n     Exemples: 5min recup, 2min repos, 90s recovery\n   • Structure complexe de l'entraînement\n     Exemples: pyramide, série, 2 blocs\n\n💡 SUGGESTIONS:\n   • Essayez: 'over-under 8x3min 95-105% FTP avec 2min recup'\n   • Spécifiez l'intensité: 'tempo', 'VO2max', '85%'\n   • Indiquez la récupération: '5min recup', '2min repos'\n   • Précisez la structure: 'pyramide', 'série', 'blocs'

🔍 ANALYSE DES ENTITÉS:
   ❌ intensity: Zone d'intensité ou pourcentage
      Exemples: VO2max, tempo
   ❌ recovery: Temps de récupération entre intervalles
      Exemples: 5min re

{'query': 'over-under 8x3min FTP',
 'confidence': 0.6699999999999999,
 'score_components': {'exact_match': 0,
  'zone_match': 0.44999999999999996,
  'duration_match': 0.48,
  'repetition_match': 0.6,
  'structure_match': 1.8},
 'corpus_match': {'id': 30,
  'description': 'over-under 8x3min 95-105% FTP avec 2min recup',
  'zone': 'Threshold',
  'structure': {'type': 'over_under',
   'reps': 8,
   'duration': 3,
   'low': 95,
   'high': 105,
   'recovery': 2},
  'power_zone': 4,
  'intensity': 100,
  'complexity': 'complex',
  'variations': ['over under 95-105%', '8x3min over-under', 'sur-sous seuil'],
  'is_variation': nan},
 'is_valid': False,
 'message': "⚠️ Correspondance insuffisante (confiance: 0.670)\\n\\n🔍 ENTITÉS MANQUANTES:\\n   • Zone d'intensité ou pourcentage\\n     Exemples: VO2max, tempo, seuil\\n   • Temps de récupération entre intervalles\\n     Exemples: 5min recup, 2min repos, 90s recovery\\n   • Structure complexe de l'entraînement\\n     Exemples: pyramide, série, 2 